**背景:**

承接demo_v1，完善上一篇被简化的部分。逐步代码模块化。

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("version of tensorflow:", tf.__version__)

version of tensorflow: 2.4.1


# 预备

## mask & padding 

可以参考[官方guide](https://www.tensorflow.org/guide/keras/understanding_masking_and_padding#passing_mask_tensors_directly_to_layers).

对不等长的序列需要padding，一般是末尾用0补齐。

In [2]:
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

# tensorflow 2.12使用下面的代码
# padded_inputs = tf.keras.utils.pad_sequences(
#     raw_inputs, padding="post"
# )
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs, padding="post"
)
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


对序列做embedding，设置mask_zero=True，查看mask的效果。  

In [3]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)


2023-05-28 23:51:11.721482: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-28 23:51:11.721835: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**备注：**  
1. 在使用tf.keras的Sequence或者Functional API时，mask layer或者指明了mask的embedding layer的下游，只要支持mask都会自动使用这个信息。
2. 可以认为embedding是mask的生产者，它实现了compute_mask方法供调用；rnn或者lstm等是mask的消费者，他们的__call__方法里支持mask参数，可以手动传进去。 

  
**回到DIN的实现上，sequence特征涉及embedding之后的pooling操作，以及attention部分，都需要处理mask的问题。**

## 规范feature column参数

### dense feature 
这类特征可以直接作为模型的输入，或者先[分桶](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Discretization)然后再embedding。

In [4]:
# 如果不做处理
age = {
    "name": "age",
    "dtype": "float32",
    "dim": (1,) # 1维
}
view_cnt = {
    "name": "view_cnt",
    "dtype": "float32",
    "dim": (100,) # 100维   
}

# 如果需要分桶再embedding
age = {
    "name": "age",
    "dtype": "float32",
    "dim": (1,), # 1维
    
    "use_bucket": True,
    "bin": [20, 30, 40, 50, 60], # 若use_bucket=False以下可以不填
    "emb_name": "age_emb",
    "emb_dim": 8
}

### sparse feature
这类特征可以直接embedding，但是如果取值过多也可以先hash到有限个取值上。参考[这里](https://www.tensorflow.org/guide/keras/preprocessing_layers#applying_the_hashing_trick_to_an_integer_categorical_feature)。

In [5]:
iid = {
    "name": "iid",
    "dtype": "bytes",
    "dim": (1,), 
    
    "vocab_size": 100000,
    "use_hash": True, # 若use_hash=False，hash_size可以不填
    "hash_size": 10000,
    "emb_name": "iid_emb",
    "emb_dim": 1000    
}

### seq/multi sparse feature
这类特征dim>1，可能是sequence，也可以不是，eg：tag list。和sparse feature相比，需要多定义max_len, combiner(pooling参数), is_sequence。

In [6]:
view_iid = {
    "name": "iid",
    "dtype": "bytes",
    "dim": (1,), 
    
    "vocab_size": 100000,
    "use_hash": True,
    "hash_size": 10000, # 若use_hash=False，hash_size可以不填
    "emb_name": "iid_emb",
    "emb_dim": 1000,
    
    "max_len": 100,
    "use_padding": True, # 如果输入不定长，则需要设置为True
    "combiner": "mean",
    "is_sequence": True     
}

## 定义模型结构参数

In [7]:
model_params = {
    "user_feat": [],
    "item_feat": [],
    "mlp_hidden_size": [128, 64, 64],
    "use_bn": True,
    "din": ["view_iid|iid", "view_cid|cid"], # 这里必须是seq特征｜item特征 
    "din_hidden_size": [64, 32]
}

# 代码模块化

## 定义feature column

## 自定义mlp结构

## 自定义din结构

## sum up